In [ ]:
# !python -m spacy download en_core_web_md
# !pip install swifter

In [4]:
import os
import nltk
curentdirectory='.'
os.chdir(curentdirectory)
nltk.data.path.append(curentdirectory+'stopwords')
nltk.data.path.append(curentdirectory+'wordnet')

In [5]:
!unzip wordnet.zip
!unzip stopwords.zip

Archive:  wordnet.zip
  inflating: wordnet/corpora/wordnet/adj.exc  
  inflating: wordnet/corpora/wordnet/index.noun  
  inflating: wordnet/corpora/wordnet.zip  
  inflating: wordnet/corpora/wordnet/data.adv  
  inflating: wordnet/corpora/wordnet/index.sense  
  inflating: wordnet/corpora/wordnet/README  
  inflating: wordnet/corpora/wordnet/noun.exc  
  inflating: wordnet/corpora/wordnet/verb.exc  
  inflating: wordnet/corpora/wordnet/citation.bib  
  inflating: wordnet/corpora/wordnet/LICENSE  
  inflating: wordnet/corpora/wordnet/data.noun  
  inflating: wordnet/corpora/wordnet/index.adj  
  inflating: wordnet/corpora/wordnet/adv.exc  
  inflating: wordnet/corpora/wordnet/data.adj  
  inflating: wordnet/corpora/wordnet/index.verb  
  inflating: wordnet/corpora/wordnet/index.adv  
  inflating: wordnet/corpora/wordnet/cntlist.rev  
  inflating: wordnet/corpora/wordnet/lexnames  
  inflating: wordnet/corpora/wordnet/data.verb  
Archive:  stopwords.zip
  inflating: stopwords/corpora/sto

In [7]:
import spacy
import os
import nltk
import string
import pandas as pd
import numpy as np
import warnings
nlp=spacy.load("en_core_web_sm")
import re

In [8]:
from nltk.stem import PorterStemmer
ps=PorterStemmer()
import swifter
stopword=nltk.corpus.stopwords.words('english')
from nltk.stem.wordnet import WordNetLemmatizer

### Preprocessing - remove punctuation, lemmatization and stemming

In [9]:
def remove_punct(text):
  text="".join([char for char in text if char not in string.punctuation])
  return text
def tokenization(text):
  text=re.split('\W+',text)
  return text
def remove_stopwords(text):
  text=tokenization(text)
  lemmatizer=WordNetLemmatizer()
  text=[ps.stem(lemmatizer.lemmatize(word)) for word in text if word not in stopword]
  return text

In [10]:
def clean_data(x):
  x=x.encode('ascii','ignore').decode()
  x=x.lower().strip()
  x=re.sub('https*\S+',' ',x)
  x=re.sub('[%s]' % re.escape(string.punctuation),' ',x)
  x=remove_punct(x)
  x=remove_stopwords(x)
  if len(x)>1:
    x=' '.join(x)
  else:
    x=None
  return x

### For training take only those intents which have more than 5 utterances. for test data take intents which are present in the train data.

In [25]:
df_train=pd.read_excel('hiri-raw-train.xlsx')
df_test=pd.read_excel('hiri-raw-test.xlsx')
df_train=df_train[(df_train.groupby('intent').transform('nunique')>3).values]
df_test=df_test[df_test.intent.isin(df_train.intent)]

### Train and predict from a SVM model

In [26]:
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score

def train_svm(df,min_df,max_df,ngram_range):
  df.dropna(inplace=True)
  df=df[(df['label']!='nan')&(df['label']!='unknown')]
  df=df[['text_cleaned','label']]
  dd=df.iloc[0].to_dict()
  # for i in dd.keys():
  #   dd[i]=['']
  # dd['label']=['Others']
  # df=pd.concat([df,pd.DataFrame(dd)])
  vec=TfidfVectorizer(min_df=min_df,ngram_range=ngram_range,max_df=max_df)
  tf=vec.fit(df.text_cleaned)
  X=tf.transform(df.text_cleaned).toarray()
  y=df.label
  tdm_s=pd.DataFrame(X,columns=vec.get_feature_names())
  print('each of the %d utterance is represented by %d features ' %(X.shape))
  sv=LinearSVC()
  sv.fit(X,y)
  #sv.intercept_[np.where(sv.classes_=='Others')[0][0]]=max(sv.intercept_)+0.00001
  pred=sv.predict(X)
  print("sv accuracy score ->", accuracy_score(pred,y)*100)
  return sv,tf

In [27]:
def predict_svm(model,tf,dff):
  dff['text_cleaned']=dff['utterance'].dropna().astype(str).apply(lambda x :clean_data(x))
  dff=dff.dropna()
  features_test=tf.transform(dff['text_cleaned'].fillna('')).toarray()
  pred=model.predict(features_test)
  proba=model.decision_function(features_test)
  dff['predictions']=pred
  dff['proba']=np.max(proba,axis=1)
  return dff

In [28]:
df_train['text_cleaned']=df_train['utterance'].apply(lambda x: clean_data(x))
df_test['text_cleaned']=df_test['utterance'].apply(lambda x: clean_data(x))

In [33]:
sv,tf=train_svm(df_train.rename(columns={'intent':'label'}),0,0.99,(1,1))
pred=predict_svm(sv,tf,df_test.rename(columns={'intent':'label'}))
accuracy_score(pred['label'],pred['predictions'])

each of the 17414 utterance is represented by 3035 features 
sv accuracy score -> 96.78419662340646


<ipython-input-27-8404da483f2f>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dff['predictions']=pred
<ipython-input-27-8404da483f2f>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dff['proba']=np.max(proba,axis=1)


0.8879361324659965

In [24]:
pred.to_csv('predictions_hiri.csv',index=None)